## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [2]:
load_dotenv(override=True)

True

In [24]:
# Let's just check emails are working for you
sender = "quoc_pham@msn.com"
recipient = "bqpham@outlook.com"

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(sender)  # Change to your verified sender
    to_email = To(recipient)  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


### Did you receive the test email

If you get a 202, then you're good to go!

If not, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with something to simply write to a flat file)

## Step 1: Agent workflow

In [4]:
instructions1 = "You are an experienced D&D dungeon master and an expert in creating D&D adventures, \
You write brief, engaging D&D encounters featuring puzzles and traps. \
"

instructions2 = "You are an experienced D&D dungeon master and an expert in creating D&D adventures, \
You write brief, balanced D&D encounters featuring monster encounters."

instructions3 = "You are an experienced D&D dungeon master and an expert in creating D&D adventures, , \
You write brief, story-driven D&D encounters featuring interesting NPCs."

In [ ]:

model = "gpt-4o-mini"

dm_agent1 = Agent(
        name="Puzzling DM",
        instructions=instructions1,
        model=model
)

dm_agent2 = Agent(
        name="Monstrous DM",
        instructions=instructions2,
        model=model
)

dm_agent3 = Agent(
        name="Characterful DM",
        instructions=instructions3,
        model=model
)

In [9]:

result = Runner.run_streamed(dm_agent1, input="Write a brief adventure no more than 1000 words.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

### Adventure Title: **The Echoes of Aeloria**

**Adventure Level:** 3-5  
**Setting:** The Sunken Ruins of Aeloria, an ancient elven city now submerged beneath a lake.

---

#### Hook

As the party travels through the quaint town of Rivermere, they hear hushed whispers about an ancient elven city, Aeloria, recently uncovered by a drought. Townsfolk speak of treasures and artifacts, but also of eerie whispers and strange occurrences. Adventurous spirits in the party may feel drawn to investigate the ruins, while more cautious members might warn against it. However, a local bard recounts tales of a fabled artifact—the Crystal of Echoes—that grants insight into the past.

#### Arrival

The riverbanks are dry, revealing toppled stone structures embedded in splintered earth. To the east, an ancient stone archway stands, its runes faint but identifiable to those knowledgeable in Arcana. As the party approaches, a sudden gust of wind rustles through the ruins, eliciting a whisper: “Seek the 

In [ ]:
message = "Write a D&D adventure no more than 1000 words."

with trace("Parallel D&D adventures"):
    results = await asyncio.gather(
        Runner.run(dm_agent1, message),
        Runner.run(dm_agent2, message),
        Runner.run(dm_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


**Title: The Echoes of Eldrin's Hollow**

### Synopsis:
Deep within the Forest of Eldrin lies an ancient ruin, said to be the remnants of a civilization that fell to darkness centuries ago. The ghosts of its truth weave an enigmatic tapestry of riddles, echoing through time. The party must navigate traps, solve puzzles, and confront their inner fears to uncover the secret behind an enchanted artifact that could either save or doom the region.

### Hook:
The players are drawn to Eldrin’s Hollow by rumors of missing villagers and a glowing artifact believed to be hidden within the ruins. An old lorekeeper in the nearby village offers them a map leading to the site and warns of “echoes” that play tricks on the mind. 

### Setting:
As the players approach Eldrin's Hollow, the air thickens with fog, muffling sound. Gnarled trees twist through a crumbling stone arch, marking the entrance to the ruin. Whispers echo as if the forest itself is alive.

---

### Encounter 1: The Hall of Whispers


In [11]:
adventure_picker = Agent(
    name="adventure_picker",
    instructions="You pick the best D&D adventure from the given options. \
Imagine you are a D&D player and fantasy fan and pick the adventure you would most like to play. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [12]:
message = "Write a D&D adventure no more than 1000 words."

with trace("Selection from dungeon masters"):
    results = await asyncio.gather(
        Runner.run(dm_agent1, message),
        Runner.run(dm_agent2, message),
        Runner.run(dm_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "D&D adventure emails:\n\n".join(outputs)

    best = await Runner.run(adventure_picker, emails)

    print(f"Best D&D adventure email:\n{best.final_output}")


Best D&D adventure email:
### The Cursed Clocktower


Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [13]:
model = "gpt-4o-mini"

dm_agent1 = Agent(
        name="Puzzling DM",
        instructions=instructions1,
        model=model
)

dm_agent2 = Agent(
        name="Monstrous DM",
        instructions=instructions2,
        model=model
)

dm_agent3 = Agent(
        name="Characterful DM",
        instructions=instructions3,
        model=model
)

In [14]:
dm_agent1

Agent(name='Puzzling DM', instructions='You are an experienced D&D dungeon master and an expert in creating D&D adventures, You write brief, engaging D&D encounters featuring puzzles and traps. ', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [15]:
sender = "quoc_pham@msn.com"
recipient = "bqpham@outlook.com"

@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(sender)  # Change to your verified sender
    to_email = To(recipient)  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [16]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4EB650220>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [18]:
tool1 = dm_agent1.as_tool(tool_name="dm_agent1", tool_description="Write a D&D adventure")
tool1

FunctionTool(name='dm_agent1', description='Write a D&D adventure', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'dm_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4EBBA8680>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [21]:
description = "Write a D&D adventure."

tool1 = dm_agent1.as_tool(tool_name="dm_agent1", tool_description=description)
tool2 = dm_agent2.as_tool(tool_name="dm_agent2", tool_description=description)
tool3 = dm_agent3.as_tool(tool_name="dm_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='dm_agent1', description='Write a D&D adventure.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'dm_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4E99845E0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='dm_agent2', description='Write a D&D adventure.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'dm_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4EC5E9760>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='dm_agent3', description='Write a D&D adventure.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['inp

## And now it's time for our Sales Manager - our planning agent

In [22]:
instructions ="You are an experienced D&D dungeon master. You use the tools given to you to generate great D&D adventures. \
You never generate D&D adventures yourself; you always use the tools. \
You try all 3 dm_agent tools once before choosing the best one. \
You pick the single best adventure and use the send_email tool to send the best adventure (and only the best adventure) to the user."

adventure_picker = Agent(
    name="Adventure Picker",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini"
)

message = "Send a D&D adventure email addressed to 'Dear DM'"


with trace("Adventure Picker"):
    result = await Runner.run(adventure_picker, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [23]:

subject_instructions = "You can write a subject any kind of email. \
You are given a message and you need to write a subject for an email that is likely to get a read and a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for any kind of email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [25]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(sender)  # Change to your verified sender
    to_email = To(recipient)  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [26]:
tools = [subject_tool, html_tool, send_html_email]

In [27]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for any kind of email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4EC5585E0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4EBC074C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to a

In [28]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [29]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='dm_agent1', description='Write a D&D adventure.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'dm_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4E99845E0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='dm_agent2', description='Write a D&D adventure.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'dm_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001B4EC5E9760>, strict_json_schema=True, is_enabled=True), FunctionTool(name='dm_agent3', description='Write a D&D adventure.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input

In [ ]:
adventure_creator_instructions = "You are an experienced D&D Dungeon Master. \
You use the tools given to you to generate D&D adventures. \
You never generate adventures yourself; you always use the tools. \
You try all 3 dm tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best D&D adventure using your own judgement of which adventure will be most enjoyable. \
After picking the adventure, you handoff to the Email Manager agent to format and send the adventure."


adventure_creator = Agent(
    name="Sales Manager",
    instructions=adventure_creator_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out an adventure mail addressed to Dear DM from AI"

with trace("Automated Adventure Creator"):
    result = await Runner.run(adventure_creator, message)

: 

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.